# Data Integration
 Purpose: This Notebook Takes all the matched entities from Drugs, Manufacturer, and Physician entity resolutions and creates the tables to be loaded into Neo4j
 
## Importing dependencies

In [1]:
import pandas as pd
import numpy as np
import pickle

## Importing Entity Resolution Lookup tables
### Manufacturer Node

In [2]:
manuf_lookup = pickle.load(open('../Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.p','rb'))
manuf_lookup[['manuf_id','manufacturer_name']].to_csv('../Nodes/Manufacturer.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Outputs_Cleanup/Manufacturer_entity_matching/Manufacturer_Matching_lookup.p'

### Drug Node

In [580]:
def string_to_ordered_list (x):
    '''
    Function that takes a string that looks like a list, and converts it to an ordered list
    '''
    ordered_list = None
    try: 
        str_list = str(x).replace('[','').replace(']','').replace(' ','').replace("'",'').split(',')
        str_list = list(map(int,str_list))
        str_set = set(str_list)
        ordered_list = sorted(list(str_set))
    except:
        ordered_list = np.NaN
    
    
    return ordered_list

In [591]:
## Importing drug matching output
drug_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine_dedupe_drug_integration/final_drug_lookup.csv')
drug_node

,drug_id,brand_name,fda_drug_id,MedD_drug_id,sunshine_drug_id
0,0,CORTISONE ACETATE,[104661],[162898],NaN
1,1,CORTEF,[170796],[159595],NaN
2,2,CORLANOR,[110500],[16549],NaN
3,3,COREG CR,[161742],[16548],NaN
4,4,CORDRAN,[35144],[118759],NaN
...,...,...,...,...,...
1657,1657,TES AIAPACK CALIBRATOR SET,NaN,NaN,TES AIAPACK CALIBRATOR SET
1658,1658,ALTERNARIA ALTERNATE ALTERNARIA TENUIS,NaN,NaN,ALTERNARIA ALTERNATE ALTERNARIA TENUIS
1659,1659,BUNAVAIL 63 MG 30COUNT BOX,NaN,NaN,BUNAVAIL 63 MG 30COUNT BOX
1660,1660,CLINPRO 5000 TOOTHPASTEVANILLA MINT,NaN,NaN,CLINPRO 5000 TOOTHPASTEVANILLA MINT


In [592]:
## Importing drug information from the deduped FDA data
openfda_drug_dedupe = pickle.load(open( "../Data/Outputs_Cleanup/FDA/Openfda_Drug_deduplicated.p", "rb" ))

## Sorting the fda_drug_id list column so that when we convert to a string, it will 
## match the fda_drug_id list column in the drug_node dataframe
openfda_drug_dedupe = openfda_drug_dedupe.applymap(set).applymap(list)
openfda_drug_dedupe.fda_drug_id = openfda_drug_dedupe.fda_drug_id.apply(sorted)

## converting the fda_drug_id string column into a sorted list column
drug_node['fda_drug_id'] = drug_node.fda_drug_id.apply(string_to_ordered_list)

## converting fda_drug_id columns in drug_node df and openfda_drug_dedupe df 
## to string from list for merging
drug_node['fda_drug_id'] = drug_node['fda_drug_id'].astype(str)
openfda_drug_dedupe['fda_drug_id'] = openfda_drug_dedupe['fda_drug_id'].astype(str)
drug_node = drug_node.merge(openfda_drug_dedupe,'left','fda_drug_id')

## Dropping id's into other datasources
# drug_node
drug_node_output = drug_node.drop(columns = ['fda_drug_id','MedD_drug_id','sunshine_drug_id','manufacturer_name'])
drug_node_output.to_csv('../Nodes/Drug_Node.csv')

In [593]:
drug_node

,drug_id,brand_name,fda_drug_id,MedD_drug_id,sunshine_drug_id,generic_name,product_type,route,substance_name,manufacturer_name,pharm_class_epc,pharm_class_pe,pharm_class_cs,pharm_class_moa
0,0,CORTISONE ACETATE,[104661],[162898],NaN,[CORTISONE ACETATE],[HUMAN PRESCRIPTION DRUG],[ORAL],[CORTISONE ACETATE],[HIKMA PHARMACEUTICALS USA INC],"[""""]","[""""]","[""""]","[""""]"
1,1,CORTEF,[170796],[159595],NaN,[HYDROCORTISONE],[HUMAN PRESCRIPTION DRUG],[ORAL],[HYDROCORTISONE],[PHARMACIA AND UPJOHN COMPANY LLC],[CORTICOSTEROID EPC],"[""""]","[""""]",[CORTICOSTEROID HORMONE RECEPTOR AGONISTS MOA]
2,2,CORLANOR,[110500],[16549],NaN,[IVABRADINE],[HUMAN PRESCRIPTION DRUG],[ORAL],[IVABRADINE HYDROCHLORIDE],[AMGEN INC],"[""""]","[""""]","[""""]","[""""]"
3,3,COREG CR,[161742],[16548],NaN,[CARVEDILOL PHOSPHATE],[HUMAN PRESCRIPTION DRUG],[ORAL],[CARVEDILOL PHOSPHATE],[GLAXOSMITHKLINE LLC],"[""""]","[""""]","[""""]","[""""]"
4,4,CORDRAN,[35144],[118759],NaN,[FLURANDRENOLIDE],[HUMAN PRESCRIPTION DRUG],[TOPICAL],[FLURANDRENOLIDE],[ALMIRALL LLC],[CORTICOSTEROID EPC],"[""""]","[""""]",[CORTICOSTEROID HORMONE RECEPTOR AGONISTS MOA]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1657,TES AIAPACK CALIBRATOR SET,nan,NaN,TES AIAPACK CALIBRATOR SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1658,1658,ALTERNARIA ALTERNATE ALTERNARIA TENUIS,nan,NaN,ALTERNARIA ALTERNATE ALTERNARIA TENUIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659,1659,BUNAVAIL 63 MG 30COUNT BOX,nan,NaN,BUNAVAIL 63 MG 30COUNT BOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1660,1660,CLINPRO 5000 TOOTHPASTEVANILLA MINT,nan,NaN,CLINPRO 5000 TOOTHPASTEVANILLA MINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Starting with the Sunshine Data
### Hospital Node

In [594]:
hospital_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/hosp_info.csv')
hospital_node.to_csv('../Nodes/Hospital_Node.csv')

### Payments Node

In [595]:
payments_node = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/payments.csv')
payments_node.to_csv('../Nodes/Payments_Node.csv')
# payments_node

### Manufacturer-Payments Edge

In [596]:
manuf_payments_edge = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/pharmCo_payments.csv')
manuf_payments_edge = manuf_payments_edge.merge(manuf_lookup,'left','Applicable_Manufacturer_or_Applicable_GPO_Making_Payment_ID')
manuf_payments_edge = manuf_payments_edge[['Record_ID','manuf_id']]
manuf_payments_edge.to_csv('../Edges/Manufacturer_Payment_Edge.csv')

### Payment-Drug Edge

In [597]:
payment_drug_edge = pd.read_csv('../Data/Outputs_Cleanup/Sunshine/record_drugs.csv')
cols = ['Record_ID','Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_1',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_2',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_3',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_4',
        'Name_of_Drug_or_Biological_or_Device_or_Medical_Supply_5'
       ]
payment_drug_edge = payment_drug_edge[cols].melt(id_vars = 'Record_ID')
payment_drug_edge.dropna(inplace=True)
payment_drug_edge = payment_drug_edge[['Record_ID','value']]
payment_drug_edge.columns = ['Record_ID','sunshine_drug_id']
drug_node[['drug_id','sunshine_drug_id']].dropna()
payment_drug_edge = payment_drug_edge.merge(drug_node[['drug_id','sunshine_drug_id']].dropna(),
                                           'left',
                                           'sunshine_drug_id')
payment_drug_edge = payment_drug_edge[~payment_drug_edge.drug_id.isna()][['Record_ID','drug_id']]

payment_drug_edge.drug_id= payment_drug_edge.drug_id.astype(int)
payment_drug_edge.to_csv('../Edges/Payment_Drug_Edge.csv')

/Users/jonahbreslow/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: DtypeWarning: Columns (22,23,24,25) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Manufacturer-Drug edge

In [598]:
openfda_manuf_dedupe = pickle.load(open( "../Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.p", "rb" ))

manuf_drug_edge = drug_node[['drug_id','manufacturer_name']]
manuf_drug_edge = manuf_drug_edge.explode('manufacturer_name')
manuf_drug_edge.dropna(inplace=True)
manuf_drug_edge = manuf_drug_edge.merge(openfda_manuf_dedupe.explode('manuf_names'),'left',left_on='manufacturer_name',right_on='manuf_names')
# manuf_drug_edge
# manuf_lookup
manuf_drug_edge = manuf_drug_edge[['drug_id','fda_manuf_id','manufacturer_name_y']].explode('fda_manuf_id')
manuf_drug_edge = manuf_drug_edge.drop_duplicates(['drug_id','manufacturer_name_y'])[['drug_id','fda_manuf_id']]
manuf_drug_edge['fda_manuf_id'] = manuf_drug_edge['fda_manuf_id'].astype(str)
manuf_drug_edge = manuf_drug_edge.merge(manuf_lookup.explode('fda_manuf_id'),'left','fda_manuf_id')
manuf_drug_edge = manuf_drug_edge[['drug_id','manuf_id']]
manuf_drug_edge.to_csv('../Edges/Manufacturer_Drug_Edge.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Outputs_Cleanup/FDA/openfda_manufacturer_deduplicated_single_manuf.p'

### Prescription Node

In [602]:
prescriptions = pickle.load(open("../Data/Outputs_Cleanup/Part_d/prescription_information.p", "rb" ))
prescriptions.head()

partD_to_drug_id = drug_node[['drug_id','MedD_drug_id']]
partD_to_drug_id['MedD_drug_id'] = partD_to_drug_id['MedD_drug_id'].apply(string_to_ordered_list)
partD_to_drug_id = partD_to_drug_id.explode('MedD_drug_id')
partD_to_drug_id.MedD_drug_id = partD_to_drug_id.MedD_drug_id.astype(float)

prescriptions = prescriptions.merge(partD_to_drug_id,'left','MedD_drug_id')


/var/folders/_7/7cv8089x2q1_td7nlck8f7lc0000gn/T/ipykernel_5854/884839904.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  partD_to_drug_id['MedD_drug_id'] = partD_to_drug_id['MedD_drug_id'].apply(string_to_ordered_list)


In [604]:
drug_node.explode('MedD_drug_id')

,drug_id,brand_name,fda_drug_id,MedD_drug_id,sunshine_drug_id,generic_name,product_type,route,substance_name,manufacturer_name,pharm_class_epc,pharm_class_pe,pharm_class_cs,pharm_class_moa
0,0,CORTISONE ACETATE,[104661],[162898],NaN,[CORTISONE ACETATE],[HUMAN PRESCRIPTION DRUG],[ORAL],[CORTISONE ACETATE],[HIKMA PHARMACEUTICALS USA INC],"[""""]","[""""]","[""""]","[""""]"
1,1,CORTEF,[170796],[159595],NaN,[HYDROCORTISONE],[HUMAN PRESCRIPTION DRUG],[ORAL],[HYDROCORTISONE],[PHARMACIA AND UPJOHN COMPANY LLC],[CORTICOSTEROID EPC],"[""""]","[""""]",[CORTICOSTEROID HORMONE RECEPTOR AGONISTS MOA]
2,2,CORLANOR,[110500],[16549],NaN,[IVABRADINE],[HUMAN PRESCRIPTION DRUG],[ORAL],[IVABRADINE HYDROCHLORIDE],[AMGEN INC],"[""""]","[""""]","[""""]","[""""]"
3,3,COREG CR,[161742],[16548],NaN,[CARVEDILOL PHOSPHATE],[HUMAN PRESCRIPTION DRUG],[ORAL],[CARVEDILOL PHOSPHATE],[GLAXOSMITHKLINE LLC],"[""""]","[""""]","[""""]","[""""]"
4,4,CORDRAN,[35144],[118759],NaN,[FLURANDRENOLIDE],[HUMAN PRESCRIPTION DRUG],[TOPICAL],[FLURANDRENOLIDE],[ALMIRALL LLC],[CORTICOSTEROID EPC],"[""""]","[""""]",[CORTICOSTEROID HORMONE RECEPTOR AGONISTS MOA]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1657,1657,TES AIAPACK CALIBRATOR SET,nan,NaN,TES AIAPACK CALIBRATOR SET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1658,1658,ALTERNARIA ALTERNATE ALTERNARIA TENUIS,nan,NaN,ALTERNARIA ALTERNATE ALTERNARIA TENUIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659,1659,BUNAVAIL 63 MG 30COUNT BOX,nan,NaN,BUNAVAIL 63 MG 30COUNT BOX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1660,1660,CLINPRO 5000 TOOTHPASTEVANILLA MINT,nan,NaN,CLINPRO 5000 TOOTHPASTEVANILLA MINT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
